EC-MS quantification with packages ixdat and spectro_inlets_quantification
=========================================================================


This tutorial is to show how to use the packages ixdat and spectro_inlets_quantification to determine sensitivity factors and used them to obtain quantiative EC-MS data.

There are three parts:

- Importing, plotting and exporting data
- Analyzing EC-MS calibrations
    - EC calibration
    - Gas calibration single analyte
    - Gas calibration multiple analytes
    - Calibration of isotopes
- Using EC-MS calibrations for quantification

Note: You need to have installed and ixdat version > 0.2.5.dev5 and spectro_inlets_quantification version > 1.2.1

You can do so using the anaconda prompt and typing:
"pip install ixdat==0.2.5.dev5"
and
"pip install -i https://test.pypi.org/simple/ spectro-inlets-quantification==1.2.1"

Finally, you'll need to download some data to analyse in this tutorial from the data repository zenodo: https://doi.org/10.5281/zenodo.8400221
Download the files and place them into the same folder as this tutorial.

Setup
=====

The following code imports ixdat and tells it to use the external quant package. This causes ixdat to import spectro_inlets_quantification. Both packages print their versions when they are imported.

In [ ]:
import ixdat


ixdat.plugins.activate_si_quant()

Later, when saving calibrations, the package will need to know which directory to use. We set that here to be the same folder as this jupyter notebook:

In [ ]:
si_quant = ixdat.plugins.si_quant
si_quant.QUANT_DIRECTORY = "."

Importing, plotting, and exporting
===========================

First, we import and plot the dataset we would like to calibrate. 

In [ ]:
from ixdat import Measurement

meas = Measurement.read(r"./2022-07-19 10_02_32 HER_OER_calibration.tsv", reader="zilien")

meas.plot_measurement()

That was a lot. Let's zoom in on an interesting part.

Tip: if you're using a backend for matplotlib that returns scalable plots (not Jupyter notebooks), you can easily zoom into your plot to find the timestamps at start and end of the part you find interesting. A left mouse click into the plot will print the time where the mouse is pointing to. A right click at a different point following a left click will print the start and end as well as length of the timespan in between. 

In [ ]:
meas.plot_measurement(tspan=[5000, 5750])

# help(meas.plot)  # This shows a full list of customization arguments to the plot function

The EC data looks a bit noisy - let's use the EC data as saved by ECLab here instead. First we import the MS data only from the Zilien data file:

Note, if you use Zilien vs. 2.6.0 or higher, Zilien will export the same data as saved by ECLab, so this will no longer be necessary.

In [ ]:
ms_meas = Measurement.read(r"./2022-07-19 10_02_32 HER_OER_calibration.tsv", reader="zilien", technique="MS")
ms_meas.replace_series("Ewe/V", None)
ms_meas.replace_series("I/mA", None)

Then we import the electrochemistry data from the text files exported by ECLab:

In [ ]:
ecdata_meas = Measurement.read_set(r".", suffix=".mpt", reader="biologic")

And combine everything:

In [ ]:
meas_combi = ms_meas + ecdata_meas

meas_combi.plot()
meas_combi.plot(tspan=[5000, 5750])

Now that looks better! Let's export that bit so we can share it with someone without exceeding an attachment limit:

In [ ]:
meas_part = meas_combi.cut(tspan=[5000, 5750])
# print(meas_part.grab("selector"))
meas_part.export("the_good_bit.csv")

ixdat can of course read the files it exports:

In [ ]:
meas_loaded = Measurement.read("the_good_bit.csv", reader="ixdat", technique="EC-MS")

meas_loaded.plot_measurement()

Calibrations
==========

EC-MS calibration
-------------------------

Now, we analyze EC-MS calibration measurements.
While ixdat comes with some basic functions for quantification, there is a more powerful package available: spectro_inlets_quantification. We will be using this package as a plugin to ixdat in this tutorial (this means that instead of using the native ixdat functions, calling the same methods will use methods from spectro_inlets_quantification instead).

(Note: The main difference between the native ixdat methods and those available in spectro_inlets_quantification is that the latter is able to handle gas mixtures. When calculating the gas flux through the capillary using the capillary equation, it will consider how properties change with composition - such as the effective mass and the viscosity - the latter changing non-linearly with composition. Additionally, it will also consider overlapping mass fragments and calculate the flux based on a matrix of sensitivity factors, rather than just looking at the primary mass of each molecule.)

So, let's get started by activating the plugin which will import the package automatically:


In [ ]:
ixdat.config.plugins.activate_si_quant()

Before we go into MS calibration, let's first calibrate the electrochemisty data. We know the geometric area of the electrode, so we can normalize the current: it's a 5mm diameter disk, area = 0.196 cm^2
We used an RHE reference electrode, so we assume that the potential difference between our reference electrode and the RHE potential is zero. We did not determine the Ohmic drop, but we will assume that it was 0 (even though that is not entirely correct), to demonstrate how we can calibrate for it. 

In [ ]:
meas.calibrate(RE_vs_RHE=0, A_el=0.196, R_Ohm=0)
# select the first hydrogen calibration and plot with calibrated EC data
meas.plot_measurement(tspan=[0,4900])

There is some data here before the hydrogen calibration measurement starts; let's shift t=0 to the right, to when the CPs start. Note that if you re-run this part, the time will be shifted further every time. Outcomment it if you do not want to move the start further.

In [ ]:
meas.tstamp += 1700

The raw data is easily plotted with the plot_measurement function, which doesn't only create a figure, but also returns
a list of three axes for (i) ms, (ii) potential, and (iii) current. See ``help(meas.plot_measurement)`` for customization. 

We will be adding to these axes later to indicate the timespans used for the calibrations.

In [ ]:
axes_a = meas.plot_measurement(tspan=(0, 3200))

Now, this is a standard EC-MS plot, which is in itself not bad. Still, sometimes one might like to change something about it, let's say the size of the axis labels, the position of the legend, or the dimensions of the figure. To do this, we can name the figure by getting it via the first axis, which then allows us to modify the plot like any regular matplotlib plot. This only works if the plot has not been printed yet, so we need to define ``axes_a`` again at the beginning. Let's make the plot wider, as an example:

In [ ]:
axes_a = meas.plot_measurement(tspan=(0, 3200))
fig_a = axes_a[0].get_figure()  # name the figure by getting it via the first axis.
fig_a.set_figwidth(
    fig_a.get_figwidth() * 2.5
)  # make it 2.5 times as wide as it is tall

Great! Now we're ready to use this data to calibrate for H2 at m/z=2. For this we use the ixdat method ``ecms_calibration_curve``, which automatically selects and integrates M2 signal and electrochemical current in the given timespans. 

In [ ]:
# redefine this axis, so we can use it here (since Jupyter doesn't update already printed plots).
axes_a = meas.plot_measurement(tspan=(0, 3200)) 
# calculate the calibration factor
cal_point_H2, ax_b = meas.ecms_calibration_curve(
    mol="H2",
    mass="M2",
    n_el=-2, # remember to use the correct sign: minus for reduction reactions, plus for oxidation reactions
    tspan_list=[(450, 500), (1000, 1050), (1550, 1600), (2200, 2250), (2750, 2800)],  # timespans of H2 evolution
    tspan_bg=(0, 100),
    ax="new",
    axes_measurement=axes_a, # to highlight the integrated areas on the plot defined above
    return_ax = True # if True, returns the calibration curve axis as a second element
) 
# NOTE, it uses and highlights the electrochemical current, not the calculated current density - this will therefore look a bit odd in the plot

Because we asked for ``return_ax``, two arguments are returned:
The first, which we call ``cal_point_H2``, is a ``CalPoint`` for H2.
The second, which we call ``ax_H2``, is the axis where it plots the calibration curve.
As we asked for ``axes_measurement``, to be plotted on ``axes_a``, the areas that were integrated are highlighted on that axes.

The attribute ``cal_point_H2.F`` is the slope of the calibration curve, which is the sensitivity factor in C/mol.
(Note that this calibration_curve works with integrals rather than simply the rates. The latter is not yet implemented in ixdat.)

In [ ]:
print(cal_point_H2)  # prints: CalPoint(mol='H2', mass='M2', F=0.3378605863775754, 
# f=None, F_type='internal', precision=None, background_signal=None, background_std=None, 
# description=None, date='23H10', setup=None, internal_conditions=None, external_conditions=None)

# save the figure:
fig_b = ax_b.get_figure()
fig_b.savefig("hydrogen_calibration.png")  # you can use eg .svg instead for vector graphics.

Tip: instead of selecting the time spans manually, you can also use the option of passing a ``selector_name (str)`` (Name of selector which identifies the periods of steady electrolysis for automatic selection of timespans of steady electrolysis. E.g. ``"selector"`` or ``"Ns"`` for biologic EC data) and a ``selector_list`` instead of the ``tspan_list``. Note that you need to have information on the different parts of the EC data in your measurement data to take advanatge of this. If you used a Zilien version < 2.6.0 when running the measurement, this requires that you import the EC data from the ECLab. Here, we can use the data imported to ``meas_combi``.

First, let's figure out what we should pass to selector list. To this end, we can plot the selector we want to use in our regular EC-MS plot, for example instead of the current: 

In [ ]:
axes_a2 = meas_combi.plot_measurement(tspan=(0, 4700), J_name="selector")
axes_a2[3].set_ylim(5,15)

In [ ]:
# redefine this axis, so we can use it here (since Jupyter doesn't update already printed plots).
axes_a2 = meas_combi.plot_measurement(tspan=(1900, 4700))
# calculate the calibration factor
cal_point_H2_selector, ax_b2 = meas_combi.ecms_calibration_curve(
    mol="H2",
    mass="M2",
    n_el=-2, # remember to use the correct sign: minus for reduction reactions, plus for oxidation reactions
    selector_name="selector",
    selector_list=[6,8,10,12,14],  # sections of H2 evolution
    tspan_bg=(1800, 1900),
    ax="new",
    axes_measurement=axes_a2, # to highlight the integrated areas on the plot defined above
    return_ax = True # if True, returns the calibration curve axis as a second element
)
print(cal_point_H2_selector)  # prints the CalPoint object

Gas calibration
-------------------------

Now, the gas calibration works a bit differently than the EC-MS calibration, as it relies on knowing the flux of molecules into the MS for the inlet used. As mentioned above, the plugin package we're using is particlarly powerful for this type of measurements. Note that the syntax here is quite a bit different from how you would use similar methods native to ixdat (but this one is better, so no need to worry about it). 

Single gas standard
-------------------

Let's import the calibration data first:

In [ ]:
meas_gascal_H2 = Measurement.read(r"./2022-07-20 11_24_43 gas_cal_high_H2.tsv", reader="zilien", 
                                 technique="MS")

meas_gascal_H2.plot()
# We don't need the first 1500s of the measurement, where the signals fluctuate a bit while pumping and flushing the gas lines
meas_gascal_H2 = meas_gascal_H2.cut(tspan=[1500, 6000])
meas_gascal_H2.plot_measurement()

Most often, we will be using a standard Spectro Inlets Chip, and be operating at (or close to) standard conditions. 

And if we are using a non-standard chip or a different capillary inlet to a mass spectrometer, we can also pass the non-standard chip dimensions, by first creating a ``Chip`` object.

Note: This is not necessary for the standard Spectro Inlets chips, we are just showing it here for completeness sake.

In [ ]:
my_chip = ixdat.plugins.si_quant.Chip()

print(f"chip capillary is {my_chip.l_cap} m")

If we want to change the pressure or temperature at which to calculate the gas flux, we can do this through the `Chip`.

(Note: spectro_inlets_quantification stores the pressure and temperature in a ``Medium``, wich is a Singleton, i.e. you don't need to pass it to other functions, when set once as below, it is available to the whole package already.)

In [ ]:
my_chip.p = 1e5   # one bar in [Pa]
my_chip.T = 298.15   # 25 deg C in [K]

Alright, we are all set up. For a singe point calibration of a pure gas (the make-up gas) and we can use the method ``gas_flux_calibration``:

In [ ]:
cal_He = meas_gascal_H2.gas_flux_calibration(mol="He",  # the molecule to calibrate
    mass="M4",  # the mass to calibrate at
    tspan=[1900, 2000], # the timespan to average the signal over
                                            )
print(cal_He)

Note: When using ixdat's native methods, you can also use the method ``gas_flux_calibration_curve``, to which you can pass several timespans corresponding to different gas compositions, i.e. you will get a multi-point calibration in a similar way as from the EC calibration. However, this method relies on the calculation of the flux of the carrier gas through the capillary and assumes that this flux will not be affected by the change in gas composition due to the analytes. This assumption likely holds as long as the gas concentration is <10% (better <1%), but is ideally only used for small analyte concentrations. This function has not been translated to use the spectro_inlets_quantification plugin, which will allow it to consider the influence of the analyte on the carrier gas. This will hopefully be done in the near future. 

If we have a more complex gas composition (e.g. a calibration gas containing 1 or more analyte gases), we can use the method ``multicomp_gas_flux_calibration``. It will take care of the changed flux properties of a gas mix compared to single gases based on the composition you pass.

In [ ]:
cal_H2_2 = meas_gascal_H2.multicomp_gas_flux_calibration(
    gas={"He": 0.999, "H2":10e-3},
    mol_list=["He", "H2"],  # the molecules to calibrate
    mass_list=["M4", "M2"],  # the masses to calibrate at
    tspan=[2900, 3100], # the timespans to average the signal over
    gas_bg="He", # the background gas
    tspan_bg=[1900, 2000], # the timespan for background subtraction (optional)
)
print(cal_H2_2)

Multiple gas standard
-------------------

Sometimes we have more than one gas in our calibration data. We can use the same method for this as above, but let's import the multi-gas calibration data first:

In [ ]:
meas_gascal_multi = Measurement.read(
        r"./2023-05-15 14_42_57 mix_cal_gas_glass_slide.tsv",
        technique="MS",
    )
meas_gascal_multi.plot_measurement(legend=None) # not show the legend because it's very long
# We don't need the first 60000s of the measurement
meas_gascal_multi = meas_gascal_multi.cut(tspan=[60000, 71000])
meas_gascal_multi.plot_measurement(legend=None)

Ok, let's calibrate. It works just like before, except we add more gases to the list this time.

In [ ]:
cal_Ar = meas_gascal_multi.gas_flux_calibration(mol="Ar", mass="M40", tspan= [60695, 61873])
print(cal_Ar)

cal_multi = meas_gascal_multi.multicomp_gas_flux_calibration(
    gas={"Ar": 1, "CO2": 1140e-6, "O2": 1053e-6, "H2": 931e-6, "C2H4": 877e-6, "CO": 991e-6},
    mol_list=["CO2", "O2",  "H2", "C2H4", "CO"],  # the molecules to calibrate
    mass_list=["M44", "M32",  "M2", "M26", "M28"],  # the masses to calibrate at
    tspan=[70076, 70830], # the timespans to average the signal over
    gas_bg="Ar", # the background gas
    tspan_bg=[60695, 61873], # the timespan for background subtraction (optional)
)
print(cal_multi)

Combining calibration results
=============================

si_quant Calibration and CalPoint objects can be combined using the `+` operator:

In [ ]:
calibration = cal_multi + cal_Ar + cal_He
# NOTE, there is a bug that causes this to crash when adding cal_H2_2! 
# We will figure this out later. For now, no problem. The calibration will use 
# The value for H2 in cal_multi and the value for He in cal_He.

Adding isotopes to the calibration
-------------------------------

If you work with isotope labelled gas, you can translate the sensitivity factors from one isotope to others. If we want to quantify for oxygen species containing 18O, this would look like this:

In [ ]:
calibration.add_isotopes({"CO2": ("M44", ["M46", "M48"]), "O2": ("M32", ["M34", "M36"]), "CO": ("M28", ["M30"])})
print(calibration)

Visualizing the calibration
-----------------------------------

The `Calibration` object has methods for analyzing the sensitivity factors it contains. A simple way to get an overview of them is to plot all the sensitivity factors vs m/z:

In [ ]:
calibration.plot_as_spectrum()

Saving the calibration
-------------------------------
Also, we might want to use the calibration for other measurements later, so let's save the calibration.

The following code saves it!

Remember, at the start of this tutorial, we set the directory to save the calibration to the current folder. Next to this jupyter notebook, you should now see a folder called "calibrations". In there is a file with the extension ".yml". This is a text file, which you can read with any plain text editor (for example Notepad).

In [ ]:
calibration.save("my_tutorial_calibration") 

Applying a calibration to your dataset
------------------------------------------------------

In the following we will explore how to apply the EC-MS calibration we determined above to a dataset. There are several ways of how you can pass a calibration to an ``ECMSMeasurement`` objects. We will only address two ways here.

Let's start by loading the calibration object that we saved earlier:

In [ ]:
loaded_cal = ixdat.plugins.si_quant.Calibration.load(
    "my_tutorial_calibration"
)
print(loaded_cal)

In [ ]:
good_bit = Measurement.read(
    "the_good_bit.csv", reader="ixdat", technique="EC-MS"
)
good_bit.plot_measurement()

And apply the calibration on the data.
When using the spectro_inlets_quantification plugin, this is done in a special way. What we actually set is a "quantifier", which apply the calibration to calculate molecular fluxes from MS data. We also have to tell it which m/z values to use (data for which should of course be in the Measurement).

In [ ]:
good_bit.set_quantifier(
    calibration=loaded_cal,
    mol_list=["H2", "O2"],
    mass_list=["M2", "M32"],
)

# Check the MS calibration most recently added to the data
print(good_bit.quantifier.calibration)

good_bit.plot_measurement()
# By default, it still plots uncalibrated data. But now we can ask to plot molecular fluxes! (next cell)

As we only provide the calibration for some of the masses, the standard plot is still showing uncalibrated MS data. If we pass the list of molecules where we do have a calibration, then ``plot_measurement()`` will plot the calibrated fluxes instead.

In [ ]:
good_bit.plot_measurement(mol_list=["H2", "O2"])

You can also get the fluxes with the `grab_flux` method. It works the same as `grab` but takes a molecule name as the input:

In [ ]:
from matplotlib import pyplot as plt


t, n_dot_H2 = good_bit.grab_flux("H2", tspan=[5050, 5100])

fig, ax = plt.subplots()

ax.plot(t, n_dot_H2, "b")
ax.set_xlabel("time / [s]")
ax.set_ylabel("my calibrated H2 signal / [mol/s]")

Alternatively, if we calculate the calibration factors in the same ``ixdat`` session as our data treatment, we can also directly apply the calibration without having to create the calibration object first. In this way, we can also make sure that we use a specific calibration factor if we have determined several for one analyte.

In [ ]:
# Lets import the file again, so we can be sure there's no calibration attached yet
good_bit2 = Measurement.read(
    "the_good_bit.csv", reader="ixdat", technique="EC-MS"
)
# Add the EC calibration first
good_bit2.calibrate(RE_vs_RHE=0, A_el=0.197)

In [ ]:
# Let's calibrate using the EC-MS calibration for hydrogen and the gas calibration for oxygen
good_bit2.set_quantifier(
    calibration=loaded_cal,
    mol_list=["H2", "O2"],
    mass_list=["M2", "M32"],
)

# Check the calibration most recently added to the data
print(good_bit2.calibration_list[0].ms_cal_results)




# And plot it once again:
good_bit2.plot_measurement(mol_list=["H2", "O2"])

Plotting calibrated EC-MS data
=========================

Now that we've calibrated the data, we want to make some nice plots. In the sections below, we will explore some ways of using ixdat's plotter and modify the plots using matplotlib.

EC-MS plot calibrated gases using two y axes
-----------------------------------------------------------------

In [ ]:
meas_a = good_bit2.cut(tspan=[5035, 5200]) #cut the data to the interesting section


# NOTE: As of now, the quantifier is not kept when cutting a measurement, so we set it again here. 
meas_a.set_quantifier(good_bit.quantifier)  # we apply the same quantifier to the cut measurement
# This will be fixed in the next release, so that setting the quantifier again is not necessary.

meas_a.tstamp += meas_a.t[0]  #to get the figure showing the time starting at 0

axes_a = meas_a.plot_measurement(
    mol_lists=[
        ["H2"],  # left y-axis
        ["O2"],  # right
    ],
    tspan_bg=[
        None,
        [0, 20],
    ],  # [left, right] y-axes
    unit=["nmol/s", "pmol/s/cm^2"],  # [left, right] y-axes
    logplot=False,
    legend=False, # e.g. if you want to add labels to the figure manually later
)
axes_a[0].set_ylabel("cal. sig. / [nmol s$^{-1}$]")
axes_a[2].set_ylabel("cal. sig. / [pmol s$^{-1}$cm$^{-2}$]")

fig_a = axes_a[0].get_figure()
fig_a.savefig("fig_a.png")

 --- Cyclic voltammetry MS plot of two of the cycles ---

In [ ]:
meas_b = good_bit2.as_cv() #by defining it as cv, meas_b.plot() would automatically plot vs potential
meas_b.set_quantifier(good_bit2.quantifier)  # again, this will not be necessary in the next release

#as soon as it's CV you can index by "cycle"
meas_b.redefine_cycle(start_potential=0.4, redox=1)
#now you can choose where a cycle starts, here defined as potential = 0.4, anodic/oxidizing = 1 (or True), cathodic/reducing = 0 (or False)
#probably the syntax is inspired by biologic data, but it actually has nothing to do with the old biologic data
#initial cycle is 0, first time the set condition is met, starts cycle 1 etc.


# Again, because the quantifier does not copy over automatically during selection, 
# we would need to reset the quantifier for each cycle selection. The following function
# defines a shortcut to do so:
def select_cycle(measurement, cycle_number):
    """Select a cycle from the measurement and apply the quantifier"""
    cycle = measurement[cycle_number]
    cycle.set_quantifier(measurement.quantifier)
    return cycle


axes_b = select_cycle(meas_b, 2).plot_vs_potential(
    mol_list=["H2", "O2"],
    unit="pmol/s/cm^2",
    logplot=False,
    legend=False,
    remove_background=True, #should remove the minimum value from the cut dataset, but it is possible somewhere to actually 
    # the background (possibly where you define the dataset)
)
select_cycle(meas_b, 3).plot_vs_potential(
    mol_list=["H2", "O2"],
    unit="pmol/s/cm^2",
    logplot=False,
    legend=False,
    remove_background=True,
    axes=axes_b, #to reuse the axis for co-plotting two separate parts of the measurement
    linestyle="--",
)
axes_b[0].set_ylabel("cal. sig. / [pmol s$^{-1}$cm$^{-2}$]")
axes_b[0].set_xlabel(meas_b.U_name)

fig_b = axes_b[0].get_figure()
fig_b.savefig("fig_b.png")

EC-MS plot with carrier gases on right y-axis (uncalibrated) and products on left (calibrated)
-----------------------------------------------------------------------------------------------------------------------------------
Note that it is not possible to mix linear and logarithmic scales in one plot with ixdat.

In [ ]:
meas_c = good_bit2.cut(tspan=[5035, 5200])
meas_c.set_quantifier(good_bit2.quantifier)
meas_c.tstamp += meas_c.t[0]

meas_c.set_bg(
   tspan_bg=[0, 20], mass_list=["M2", "M32"]
)

axes_c = meas_c.plot_measurement(
    mol_lists = [[], ["H2", "O2"]], # left y-axis
    logplot=False,
    legend=False,
)
meas_c.plot_measurement(
   mass_list=["M4", "M28", "M40"],  # right y-axis
    #unit=["nmol/s"],  # [left, right] y-axes
    logplot=False,
    legend=False,
    axes=axes_c, #to reuse the axis for co-plotting two separate parts of the measurement
    #linestyle="--",
)
axes_c[2].set_ylabel("cal. sig. / [nmol s$^{-1}$]")
# axes_c[-1].set_ylim(bottom=-5)

fig_c = axes_c[0].get_figure()
fig_c.savefig("fig_c.png")

EC-MS plot with a third with system parameters added
-----------------------------------------------------------------------------

In some cases it can be interesting to co-plot some other data, e.g. system parameters like iongauge pressure or MFC flow rates. Let's have a look at the iongauge pressure during one of the CV cycles and plot that in a third panel of our regular EC-MS plots.

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import gridspec

# since we want to co-plot system parameters here, which were not part of the exported dataset, we need to use another version
# of our dataset # imported above, which hasn't been calibrated yet, so let's quickly calibrate that now
meas_part.set_quantifier(
    calibration=loaded_cal,
    mol_list=["H2", "O2"],
    mass_list=["M2", "M32"],
)


# cut the relevant part from a big dataset
meas_d = meas_part.cut(tspan=[5035, 5200])

# grab the iongauge pressure
ig1 = meas_d.grab("Iongauge value [mbar]")

# create a figure with 3 panels
fig_4 = plt.figure()
gs = gridspec.GridSpec(7, 1, fig_4)
ax_ms = plt.subplot(gs[0:3, 0]) # MS signals panel
ax_ms_right = ax_ms.twinx()     # right axis for the MS signals, not used here
ax_ec = plt.subplot(gs[3:5, 0]) # EC signals panel
ax_ec_right = ax_ec.twinx()     # axis for the current
ax_ig= plt.subplot(gs[5:7, 0]) # Ion Gauge panel
fig_4.set_figheight(7)
fig_4.set_figwidth(fig_4.get_figheight() * 1.25)

ax_ms.tick_params(
    axis="x", top=True, bottom=True, labeltop=True, labelbottom=False
)
ax_ec.tick_params(
    axis="x", top=True, bottom=True, labeltop=False, labelbottom=False
)
ax_ig.tick_params(
    axis="x", top=True, bottom=True, labeltop=False, labelbottom=True
)

ax_ms.set_xlabel("time / [s]")
ax_ms.xaxis.set_label_position("top")

ax_ig.set_xlabel("time / [s]")
ax_ig.set_ylabel("IG pressure / [mbar]")
ax_ms.set_ylabel("Signal / [A]")
ax_ms.set_yscale("log")


ax_ec.set_ylabel("U$_{RHE}$ / (V)")
ax_ec_right.set_ylabel("J$_{total}$ / (mA cm$^{-2}$)", color="red")


# add the MS and EC data to the correct axes using ixdat's "plot measurement"
meas_d.plot_measurement(axes=[ax_ms, ax_ec, ax_ms_right, ax_ec_right])
# this overwrites some of the axis labels, so we need to define them (again) if we want them non-standard
ax_ec.set_xlabel("")
ax_ec_right.set_xlabel("")

# add the iongauge data from the "grabbed" data series
ax_ig.plot(ig1[0], ig1[1])

Congratulations! You have reached the end of this tutorial and are now an expert in loading, calibrating and plotting EC-MS data using ``ixdat`` and ``spectro_inlets_quantification``.